In [1]:
import pandas as pd
from pandas import json_normalize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import gensim
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
ruta1= r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\Sitios.PARQUET"
ruta2=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\Scraping_sitios_google.PARQUET"
ruta3=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\user.PARQUET"
ruta4=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\Reviews_florida.PARQUET"
ruta5=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\review.PARQUET"
ruta6=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\business.PARQUET"

In [3]:
dfSitios=pd.read_parquet(ruta1)
dfScraping=pd.read_parquet(ruta2)
dfuser=pd.read_parquet(ruta3)
dfReviewsFL=pd.read_parquet(ruta4)
dfreview=pd.read_parquet(ruta5)
dfbusiness=pd.read_parquet(ruta6)

In [4]:
modelo_yelpdf = pd.merge(dfbusiness[['business_id', 'name', 'city', 'latitude', 'longitude', 'stars business']],
                               dfreview[['business_id', 'text', 'stars review', 'fecha review']],
                               on='business_id')

In [5]:
modelo_googledf = pd.merge(dfReviewsFL[['business_name', 'rating', 'city', 'gmap_id', 'date', 'text']],
                                    dfSitios[['gmap_id','latitude', 'longitude', 'avg_rating']],
                                    on='gmap_id')


In [6]:
modelo_googledf = modelo_googledf.rename(columns={
    'business_id': 'business_id',
    'business_name': 'business_name',
    'city': 'city',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'avg_rating': 'business_rating',
    'text': 'review_text',
    'rating': 'review_stars',
    'date': 'review_date'
})

In [7]:
modelo_yelpdf.rename(columns={'business_id': 'business_id', 'name': 'business_name', 'city': 'city', 'latitude': 'latitude', 'longitude': 'longitude', 'stars business': 'business_rating', 'text': 'review_text', 'stars review': 'review_stars', 'fecha review': 'review_date'}, inplace=True)


In [8]:
#hacer un append de las tablas modelo_regresion_yelpdf y modelo_regresion_googledf y guardar el resultado en un data frame de pandas llamado modelo_regresiondf y luego pasar a un archivo parquet llamado modelo_regresion.csv
dfFinal = pd.concat([modelo_yelpdf, modelo_googledf])

In [9]:
# Inicializar el analizador de sentimientos de NLTK
sid= SentimentIntensityAnalyzer()

In [10]:
#Analisis de sentimiento para el archivo review

# Retrona 0 si es una reseña negativa
# Retrona 1 si es una reseña nula o neutra
# Retrona 2 si es una reseña positiva

def analyze_sentiment(text):
    if text is None or pd.isna(text):
        return 1 
    else:
        
        sentiment_score = sid.polarity_scores(text)['compound']
        if sentiment_score < -0.05:
            return 0  
        elif sentiment_score > 0.05:
            return 2  
        else:
            return 1  

dfFinal['sentiment_analysis'] = dfFinal['review_text'].apply(analyze_sentiment)

In [11]:
filtro= dfFinal["business_name"] == "Taco Bell"
dfTacoBell=dfFinal[filtro]

In [12]:

# Convertir la columna review_date a formato numérico
dfTacoBell['review_date'] = pd.to_datetime(dfTacoBell['review_date']).dt.strftime('%Y%m%d').astype(int)

# Filtrar los datos para los restaurantes Taco Bell
taco_bell_data = dfTacoBell[dfTacoBell['business_name'].str.contains('Taco Bell', case=False)]

# Seleccionar las características y la variable objetivo
features = ['review_date', 'city']
target = 'business_rating'

# Codificar la columna 'city' utilizando one-hot encoding
encoder = OneHotEncoder(handle_unknown='ignore')
city_encoded = encoder.fit_transform(taco_bell_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
city_encoded_df = pd.DataFrame(city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con las otras características
taco_bell_data_encoded = pd.concat([taco_bell_data[['review_date']].reset_index(drop=True), city_encoded_df], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(taco_bell_data_encoded, taco_bell_data[target], test_size=0.2, random_state=42)

# Crear el modelo de regresión de bosque aleatorio
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

C:\Users\juanc.DESKTOP-LGMDQP1\AppData\Local\Temp\ipykernel_7492\2652954398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTacoBell['review_date'] = pd.to_datetime(dfTacoBell['review_date']).dt.strftime('%Y%m%d').astype(int)


Mean Squared Error: 0.18875514185720832


In [13]:
# Ejemplo de uso: predecir el business_rating para Taco Bell en los próximos años
new_data = pd.DataFrame({'review_date': [20230101, 20240101, 20250101], 'city': ['Miami', 'Orlando', 'Tampa']})

# Codificar la columna 'city' del nuevo conjunto de datos utilizando el mismo encoder
new_city_encoded = encoder.transform(new_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
new_city_encoded_df = pd.DataFrame(new_city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con la columna 'review_date'
new_data_encoded = pd.concat([new_data[['review_date']], new_city_encoded_df], axis=1)

# Asegurarse de que las columnas del nuevo conjunto de datos coincidan con las del conjunto de entrenamiento
missing_cols = set(X_train.columns) - set(new_data_encoded.columns)
for c in missing_cols:
    new_data_encoded[c] = 0
new_data_encoded = new_data_encoded[X_train.columns]

# Realizar predicciones con el modelo
new_predictions = model.predict(new_data_encoded)

print("Predicted business_rating for Taco Bell in the next years:")
for year, prediction in zip([2023, 2024, 2025], new_predictions):
    print(f"Year {year}: {prediction}")

Predicted business_rating for Taco Bell in the next years:
Year 2023: 2.21
Year 2024: 2.21
Year 2025: 1.71


In [14]:
filtro=dfFinal["city"] == "Miami"
dfFinal[filtro]

,business_id,business_name,city,latitude,longitude,business_rating,review_text,review_stars,review_date,gmap_id,sentiment_analysis
126,NaN,Rincon Caribeno,Miami,25.768021,-80.376264,4.0,I sang karaoke there people love it,5,2020-02-09,0x88d9bf280d7c90a7:0xed05e00344fbe5e2,2
127,NaN,Rincon Caribeno,Miami,25.768021,-80.376264,4.0,(Translated by Google) I like good atmosphere\...,5,2021-07-13,0x88d9bf280d7c90a7:0xed05e00344fbe5e2,2
128,NaN,Rincon Caribeno,Miami,25.768021,-80.376264,4.0,(Translated by Google) There is no food and wh...,3,2020-10-02,0x88d9bf280d7c90a7:0xed05e00344fbe5e2,0
129,NaN,Rincon Caribeno,Miami,25.768021,-80.376264,4.0,(Translated by Google) Good place\n\n(Original...,5,2018-07-27,0x88d9bf280d7c90a7:0xed05e00344fbe5e2,2
130,NaN,Rincon Caribeno,Miami,25.768021,-80.376264,4.0,(Translated by Google) Good food good dervish\...,5,2016-12-17,0x88d9bf280d7c90a7:0xed05e00344fbe5e2,2
...,...,...,...,...,...,...,...,...,...,...,...
15639,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,1,2020-04-01,0x88d9bf290ee132e7:0x771f3f6950e264db,1
15640,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2021-03-31,0x88d9bf290ee132e7:0x771f3f6950e264db,1
15641,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2020-09-14,0x88d9bf290ee132e7:0x771f3f6950e264db,1
15642,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2020-09-28,0x88d9bf290ee132e7:0x771f3f6950e264db,1
